In [1]:
%pwd

'd:\\IMDB-Dataset\\notebook'

In [2]:
import os
os.chdir('../')
%pwd

'd:\\IMDB-Dataset'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path: Path

In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])
        data_tranformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_tranformation_config

In [ ]:
import polars as pl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pl.read_csv("artifacts/data_ingestion/IMDB_Dataset.csv")
df = df.drop_duplicates()

In [ ]:
reviews = df['review'].to_list()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews)
dense_X = X.toarray()
df_vectorized = pl.DataFrame(dense_X)

In [ ]:
le = LabelEncoder()
encoded_sentiment = le.fit_transform(df['sentiment'].to_list())
df_vectorized = df_vectorized.with_columns(
    pl.Series("sentiment", encoded_sentiment)
)

In [14]:
print(df_vectorized.head())

shape: (5, 35_214)
┌──────────┬──────────┬──────────┬──────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ column_0 ┆ column_1 ┆ column_2 ┆ column_3 ┆ … ┆ column_352 ┆ column_352 ┆ column_352 ┆ column_35 │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆   ┆ 10         ┆ 11         ┆ 12         ┆ 213       │
│ i64      ┆ i64      ┆ i64      ┆ i64      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│          ┆          ┆          ┆          ┆   ┆ i64        ┆ i64        ┆ i64        ┆ i64       │
╞══════════╪══════════╪══════════╪══════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 0        ┆ 0        ┆ 0        ┆ 0        ┆ … ┆ 0          ┆ 0          ┆ 0          ┆ 0         │
│ 0        ┆ 0        ┆ 0        ┆ 0        ┆ … ┆ 0          ┆ 0          ┆ 0          ┆ 0         │
│ 0        ┆ 0        ┆ 0        ┆ 0        ┆ … ┆ 0          ┆ 0          ┆ 0          ┆ 0         │
│ 0        ┆ 0        ┆ 0        ┆ 0        ┆ … ┆ 0          ┆ 0        

In [12]:
import os
import pandas as pd
from src import logger
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [13]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        try:
            # Read the dataset
            df = pd.read_csv(self.config.data_path)

            # Drop duplicates
            df = df.drop_duplicates()
            logger.info(f"Shape after dropping duplicates: {df.shape}")

            # Encode the 'sentiment' column
            le = LabelEncoder()
            encoded_last_col = le.fit_transform(df.iloc[:, -1])
            df['sentiment'] = encoded_last_col

            # Tokenize the 'review' column
            tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
            tokenizer.fit_on_texts(df['review'])
            sequences = tokenizer.texts_to_sequences(df['review'])

            # Pad the sequences
            df['review'] = list(pad_sequences(sequences, maxlen=100, padding='post'))


            # Split the data into training and test sets (75%, 25%)
            train, test = train_test_split(df, test_size=0.25, random_state=42)

            # Ensure the root directory exists
            os.makedirs(self.config.root_dir, exist_ok=True)

            # Save training and test sets to CSV
            train_path = os.path.join(self.config.root_dir, "train.csv")
            test_path = os.path.join(self.config.root_dir, "test.csv")
            train.to_csv(train_path, index=False)
            test.to_csv(test_path, index=False)

            # Log details
            logger.info("Split data into training and test sets")
            logger.info(f"Training set shape: {train.shape}")
            logger.info(f"Test set shape: {test.shape}")

            print(f"Training set shape: {train.shape}")
            print(f"Test set shape: {test.shape}")
        except Exception as e:
            logger.error(f"Error during train-test splitting: {e}")
            raise

In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-12-25 11:15:40,432: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-25 11:15:40,463: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-25 11:15:40,463: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-25 11:15:40,463: INFO: common: created directory at: artifacts]
[2024-12-25 11:15:40,543: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-25 11:15:41,055: INFO: 689007881: Shape after dropping duplicates: (3997, 2)]
[2024-12-25 11:15:44,746: INFO: 689007881: Split data into training and test sets]
[2024-12-25 11:15:44,746: INFO: 689007881: Training set shape: (2997, 2)]
[2024-12-25 11:15:44,746: INFO: 689007881: Test set shape: (1000, 2)]
Training set shape: (2997, 2)
Test set shape: (1000, 2)
